In [1]:
%AddJar http://central.maven.org/maven2/org/bitcoinj/bitcoinj-core/0.14.7/bitcoinj-core-0.14.7-bundled.jar

Starting download from http://central.maven.org/maven2/org/bitcoinj/bitcoinj-core/0.14.7/bitcoinj-core-0.14.7-bundled.jar
Finished download of bitcoinj-core-0.14.7-bundled.jar


In [2]:
import org.bitcoinj.core.{Block, Context, Transaction, Utils}
import org.bitcoinj.params.MainNetParams
import scala.util.{Failure, Success, Try}

In [3]:
def isMagic(bytes: Array[Byte], startSeq: Int): Boolean= {
            if ((bytes.size != startSeq)
                && (bytes(startSeq) & 0xFF) == 0xF9
                && (bytes(startSeq + 1) & 0xFF) == 0xbe
                && (bytes(startSeq + 2) & 0xFF) == 0xb4
                && (bytes(startSeq + 3) & 0xFF) == 0xd9)
                return true

            false
        }

isMagic: (bytes: Array[Byte], startSeq: Int)Boolean


In [5]:
def toBlockBytes(t: Array[Byte]) : Seq[Array[Byte]] = {
    var blocksAsBytes = Seq.newBuilder[Array[Byte]]
    var byteCursor = 0
    var blockCount = 0

    while (isMagic(t, byteCursor)) {

        blockCount = blockCount + 1;

        var blockSizeBytes = new Array[Byte](4)
        System.arraycopy(t, byteCursor + 4, blockSizeBytes,0,4)
        val blockSize = Utils.readUint32BE(Utils.reverseBytes(blockSizeBytes), 0)
        var blockBytes = new Array[Byte](blockSize.toInt)
        System.arraycopy(t,byteCursor + 8,blockBytes,0, blockSize.toInt);
        byteCursor = byteCursor + 8 + blockSize.toInt;
        blocksAsBytes += blockBytes
    }    
    
    blocksAsBytes.result()
} 

toBlockBytes: (t: Array[Byte])Seq[Array[Byte]]


Name: Unknown Error
Message: <console>:1: error: ':' expected but ')' found.
def g(x) = {
       ^

StackTrace: 

In [11]:
var files = sc.binaryFiles("/bitcoin-blocks/blk0000*.dat")
                .map(f => (f._1,f._2.toArray()))

files = MapPartitionsRDD[7] at map at <console>:42


lastException: Throwable = null


MapPartitionsRDD[7] at map at <console>:42

In [32]:
var blocksAsBytes = files.map(f => (f._1, Try(toBlockBytes(f._2))))

blocksAsBytes = MapPartitionsRDD[6] at map at <console>:53


MapPartitionsRDD[6] at map at <console>:53

In [35]:
var failed = blocksAsBytes.filter(f => f._2.isFailure)
                .map(f => f._1)
                .distinct

failed = MapPartitionsRDD[14] at distinct at <console>:57


MapPartitionsRDD[14] at distinct at <console>:57

In [9]:
var filesSizes = files.map(f => f._2.length)

filesSizes = MapPartitionsRDD[5] at map at <console>:43


MapPartitionsRDD[5] at map at <console>:43

In [12]:
files.count

Exception in thread "Executor task launch worker for task 2" java.lang.SecurityException: Not allowed to invoke System.exit!
	at org.apache.toree.security.KernelSecurityManager.checkExit(KernelSecurityManager.scala:147)
	at java.lang.Runtime.halt(Runtime.java:273)
	at org.apache.spark.util.SparkUncaughtExceptionHandler.uncaughtException(SparkUncaughtExceptionHandler.scala:32)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:557)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)


Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 2, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.toByteArray(ByteArrayOutputStream.java:191)
	at org.spark_project.guava.io.ByteStreams.toByteArray(ByteStreams.java:253)
	at org.apache.spark.input.PortableDataStream.toArray(PortableDataStream.scala:193)
	at $line50.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:42)
	at $line50.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anonfun$1.apply(<console>:42)
	at scala.collection.Iterator$$anon$11.next(Iterator.scala:409)
	at org.apache.spark.util.Utils$.getIteratorSize(Utils.scala:1838)
	at org.apache.spark.rdd.RDD$$anonfun$count$1.apply(RDD.scala:1162)
	at org.apache.spark.rdd.RDD$$anonfun$count$1.apply(RDD.scala:1162)
	at org.apache.spark.SparkContext$$anonf